In [1]:
!pip install pycryptodome

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----------- ---------------------------- 0.5/1.8 MB 4.2 MB/s eta 0:00:01
   ----------------------------------- ---- 1.6/1.8 MB 7.0 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 3.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import json
import uuid
import base64
import requests
from datetime import datetime, date

from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5


class CbsApiComponent:

    def __init__(self):
        secret_key = r'C:\Users\KBL\Desktop\LetterAction_Bot1\kbl-letter-action\rpa\PrivateKey_20250306124027.pem'
        self.pem_file = secret_key
        self.pvtkey = self._read_private_key()
        
    def _read_private_key(self):
        with open(self.pem_file, 'rb') as file:
            return file.read()

    def _create_signature(self, reqModel):
        TimeStamp = datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f")[:23]
        signatureModel = {
            "Model": reqModel,
            "TimeStamp": TimeStamp
        } 

        signaturebytes = json.dumps(signatureModel, seperators=(",",":")).encode("utf-8")
        privateRSAkey = RSA.importKey(self.pvtkey)
        signer = PKCS1_v1_5.new(privateRSAkey)
        digest = SHA256.new()
        digest.update(signaturebytes)
        return base64.b64encode(signer.sign(digest)).decode("utf-8"), TimeStamp
    

    def _create_request_data(sle, request_model, signature, TimeStamp, function):
        request_byte = json.dumps(request_model, seperators=(",", ":")).encode("utf-8")
        return {
            "FunctioName": function,
            "Data": base64.b64encode(request_byte).decode('utf-8'),
            "Signature": signature,
            "TimeStamp": TimeStamp,
        }
    

    def _send_request(self, request_data):
        request_data_json = json.dumps(request_data)
        client = requests.Session()

        username = 'kblrpa'
        password = 'KblRpa@123'

        url = "http://"
        client.auth = (username, password)
        headers = {"Content-Type": "application/json", "Accept": "application/json"}
        response = client.post(url, headers=headers, data=request_data_json)
        return response, request_data_json